In [ ]:
# ✅ Final Colab Notebook for Extracting Clean Contact List
!pip install pymupdf pandas openpyxl --quiet
from google.colab import files
import fitz  # PyMuPDF
import pandas as pd
import re
import os

# Upload the PDF file
uploaded = files.upload()
pdf_path = next(iter(uploaded))
doc = fitz.open(pdf_path)
text = "".join(page.get_text() for page in doc)
blocks = text.split("View More")

# Identify where real contacts start
start_index = -1
for i, block in enumerate(blocks):
    lines = [line.strip() for line in block.strip().split("\n") if line.strip()]
    content_lines = [
        l for l in lines if not any(ui in l.lower() for ui in [
            "edit search", "save search", "no active tag", "send email",
            "learn more", "mailto:", "display results", "view more", "start autopilot"])
    ]
    for j in range(len(content_lines) - 2):
        if len(content_lines[j].split()) >= 2 and any("@" in l for l in lines):
            start_index = i
            break
    if start_index != -1:
        break

# Parse valid contact blocks
final_data = []
for block in blocks[start_index:]:
    block_lower = block.lower()
    if any(x in block_lower for x in ["results per page", "display results", "/splash", "start autopilot", "view more", "mailto:"]):
        continue
    lines = [line.strip() for line in block.strip().split("\n") if line.strip()]
    name = role = company = ""
    emails, phone = [], ""
    content_lines = [line for line in lines if not any(x in line.lower() for x in [
        "edit search", "save search", "no active tag", "send email",
        "learn more", "mailto:", "display results", "view more", "start autopilot"])]
    for i in range(len(content_lines) - 2):
        if len(content_lines[i].split()) >= 2:
            name = content_lines[i]
            role = content_lines[i + 1]
            company = content_lines[i + 2]
            break
    for line in lines:
        if "@" in line and not line.lower().startswith("mailto:"):
            emails.append(line)
        elif re.search(r"\+?\d[\d\s.-]{7,}", line):
            match = re.search(r"(\+?\d[\d\s.-]{7,})", line)
            if match:
                phone = match.group(1)
    if name and (emails or phone):
        final_data.append({
            "NAME": name.strip(),
            "ROLE": role.strip(),
            "COMPANY": company.strip(),
            "MAIL 1": emails[0] if len(emails) > 0 else "",
            "MAIL 2": emails[1] if len(emails) > 1 else "",
            "CONTACT": phone.strip()
        })

# Convert to DataFrame
df = pd.DataFrame(final_data)

# Final manual fix for Fernando Moya Hernandez
idx = df[df["MAIL 1"].str.contains("fernando.moya.hernandez", na=False)].index
if not idx.empty:
    df.at[idx[0], "NAME"] = "Fernando Moya Hernandez"
    df.at[idx[0], "ROLE"] = "Chief Financial Officer"
    df.at[idx[0], "COMPANY"] = "SCALPERS"
    df.at[idx[0], "CONTACT"] = ""

# Remove junk and duplicate rows
df = df[~df.apply(lambda row: row.astype(str).str.lower().str.contains("result per page").any(), axis=1)]
df = df.drop_duplicates().reset_index(drop=True)

# Remove row 55 if exists
if len(df) > 54:
    df = df.drop(index=54).reset_index(drop=True)

# Add serial number and updated date
df.insert(0, "S. No", range(1, len(df) + 1))
df["DATE"] = "27/07/2025"

# Export to Excel
output_path = "/content/Final_Contact_List_Cleaned_Ready.xlsx"
df.to_excel(output_path, index=False)
files.download(output_path)